# Results, comparison, and suggestions for improvement
This notebook will contain a sample of results using real data sources, a comparison of results with the rule-based project that was previously conducted, and some suggestions for improvement.

The majority of the data that was used for training originates from qudt-property and qudt-unit. These files can be found the the raw datasets folder of this repo. It should be noted that the matches this program outputs are limited to the labels that can be found in the training data.

In [1]:
from main import app_load_models, user_input_loop

Using TensorFlow backend.


In [3]:
app_load_models()

## 1. Results

### 1.1 Real data

First, we'll try inputting strings from real data sources and evaluating the matches we get. 

- TBSHT01_Humidity_Temp
- Wind Speed m/s
- AvgWindDirection
- MinWindSpeed

In [2]:
user_input_loop(notebook=True)

Enter a string to predict:TBSHT01_Humidity_Temp
no abbreviation for tbsht01_
no abbreviation for humidity_


,processed_name,proper_name,suggested_unit,suggested_unit_url,url
0,relative humidity,relative humidity,NaN,NaN,NaN
1,air temperature,air temperature,Degree Celsius,http://qudt.org/vocab/unit#DegreeCelsius,http://registry.it.csiro.au/def/environment/pr...
2,subsurface temperature,subsurface temperature,NaN,NaN,NaN
3,water temperature,water temperature,Degree Celsius,http://qudt.org/vocab/unit#DegreeCelsius,http://registry.it.csiro.au/def/environment/pr...
4,sea water temperature,sea water temperature,Degree Celsius,http://qudt.org/vocab/unit#DegreeCelsius,http://registry.it.csiro.au/def/environment/pr...
5,dew point temperature,dew point temperature,Degree Celsius,http://qudt.org/vocab/unit#DegreeCelsius,http://registry.it.csiro.au/def/environment/pr...
6,carbon concentration,carbon concentration,NaN,NaN,http://registry.it.csiro.au/def/environment/pr...
7,oxygen concentration,oxygen concentration,NaN,NaN,http://registry.it.csiro.au/def/environment/pr...
8,nitrogen concentration,nitrogen concentration,Milligrams Per Litre | Milligrams Per Cubic Me...,http://registry.it.csiro.au/def/environment/un...,http://registry.it.csiro.au/def/environment/pr...
9,carbon organic concentration,carbon - organic concentration,NaN,NaN,http://registry.it.csiro.au/def/environment/pr...


,abbreviation,processed_name,proper_name,url
0,abA,abampere,Abampere,http://registry.it.csiro.au/def/qudt/1.1/qudt-...
1,pc,parsec,Parsec,http://registry.it.csiro.au/def/qudt/1.1/qudt-...
2,Pa,pascal,Pascal,http://registry.it.csiro.au/def/qudt/1.1/qudt-...
3,P / hr,pascal per hour,Pascal per Hour,http://registry.it.csiro.au/def/qudt/1.1/qudt-...
4,P / min,pascal per minute,Pascal per Minute,http://registry.it.csiro.au/def/qudt/1.1/qudt-...
5,P / s,pascal per second,Pascal per Second,http://registry.it.csiro.au/def/qudt/1.1/qudt-...
6,Pa-s,pascal second,Pascal Second,http://registry.it.csiro.au/def/qudt/1.1/qudt-...
7,MOP,pataca,Pataca,http://registry.it.csiro.au/def/qudt/1.1/qudt-...
8,Pi,pebi,Pebi,http://registry.it.csiro.au/def/qudt/1.1/qudt-...
9,dwt,penny weight,Penny Weight,http://registry.it.csiro.au/def/qudt/1.1/qudt-...


-----------------------------------------------------------
Enter a string to predict:Wind Speed m/s
no abbreviation for m/s


,processed_name,proper_name,suggested_unit,suggested_unit_url,url
0,wind speed,wind speed,Kilometer per Hour,http://registry.it.csiro.au/def/qudt/1.1/qudt-...,http://registry.it.csiro.au/def/environment/pr...
1,wind direction,wind direction,Degree Angle,http://registry.it.csiro.au/def/qudt/1.1/qudt-...,http://registry.it.csiro.au/def/environment/pr...
2,maximum wind gust,maximum wind gust,NaN,NaN,NaN
3,wind velocity,wind velocity,NaN,NaN,NaN
4,carbon concentration,carbon concentration,NaN,NaN,http://registry.it.csiro.au/def/environment/pr...
5,oxygen concentration,oxygen concentration,NaN,NaN,http://registry.it.csiro.au/def/environment/pr...
6,water temperature,water temperature,Degree Celsius,http://qudt.org/vocab/unit#DegreeCelsius,http://registry.it.csiro.au/def/environment/pr...
7,nitrogen concentration,nitrogen concentration,Milligrams Per Litre | Milligrams Per Cubic Me...,http://registry.it.csiro.au/def/environment/un...,http://registry.it.csiro.au/def/environment/pr...
8,hydrocarbon concentration,hydrocarbon concentration,NaN,NaN,http://registry.it.csiro.au/def/environment/pr...
9,phosphorus concentration,phosphorus concentration,Milligrams Per Litre,http://registry.it.csiro.au/def/environment/un...,http://registry.it.csiro.au/def/environment/pr...


...

""


-----------------------------------------------------------
Enter a string to predict:MinWindSpeed


,processed_name,proper_name,suggested_unit,suggested_unit_url,url
0,wind speed,wind speed,Kilometer per Hour,http://registry.it.csiro.au/def/qudt/1.1/qudt-...,http://registry.it.csiro.au/def/environment/pr...
1,wind direction,wind direction,Degree Angle,http://registry.it.csiro.au/def/qudt/1.1/qudt-...,http://registry.it.csiro.au/def/environment/pr...
2,maximum wind gust,maximum wind gust,NaN,NaN,NaN
3,wind velocity,wind velocity,NaN,NaN,NaN
4,carbon concentration,carbon concentration,NaN,NaN,http://registry.it.csiro.au/def/environment/pr...
5,oxygen concentration,oxygen concentration,NaN,NaN,http://registry.it.csiro.au/def/environment/pr...
6,water temperature,water temperature,Degree Celsius,http://qudt.org/vocab/unit#DegreeCelsius,http://registry.it.csiro.au/def/environment/pr...
7,nitrogen concentration,nitrogen concentration,Milligrams Per Litre | Milligrams Per Cubic Me...,http://registry.it.csiro.au/def/environment/un...,http://registry.it.csiro.au/def/environment/pr...
8,hydrocarbon concentration,hydrocarbon concentration,NaN,NaN,http://registry.it.csiro.au/def/environment/pr...
9,phosphorus concentration,phosphorus concentration,Milligrams Per Litre,http://registry.it.csiro.au/def/environment/un...,http://registry.it.csiro.au/def/environment/pr...


,abbreviation,processed_name,proper_name,url
0,min,minute sidereal,Minute Sidereal,http://registry.it.csiro.au/def/qudt/1.1/qudt-...
1,m min^-1,meter per minute,Meter per Minute,http://registry.it.csiro.au/def/qudt/1.1/qudt-...
2,ft/min,foot per minute,Foot per Minute,http://registry.it.csiro.au/def/qudt/1.1/qudt-...
3,lb/min,pound per minute,Pound per Minute,http://registry.it.csiro.au/def/qudt/1.1/qudt-...
4,kcal/min,kilocalorie per minute,Kilocalorie per Minute,http://registry.it.csiro.au/def/qudt/1.1/qudt-...
5,P / min,pascal per minute,Pascal per Minute,http://registry.it.csiro.au/def/qudt/1.1/qudt-...
6,gal/min,us gallon per minute,US Gallon per Minute,http://registry.it.csiro.au/def/qudt/1.1/qudt-...
7,mi/min,mile per minute,Mile per Minute,http://registry.it.csiro.au/def/qudt/1.1/qudt-...
8,ft^3/min,cubic foot per minute,Cubic Foot per Minute,http://registry.it.csiro.au/def/qudt/1.1/qudt-...
9,rev/min,revolution per minute,Revolution per Minute,http://registry.it.csiro.au/def/qudt/1.1/qudt-...


-----------------------------------------------------------
Enter a string to predict:xxx


The returned results are ranked by the probability the classifier thinks the result is correct. As there are hundreds of possible labels and usually only one of two data samples per label, this probability is very small and therefore sensitive. Therefore, ten results are returned as the right answer will sometimes appear a little further down the list than first place.

#### Summary of real data results

TBSHT01_Humidity_Temp
- Had a reasonably good result given that there's no label that integrates both "humidity" and "temperature". The top labels for property were "relative humidity" and "air temperature".
- Something, possibly the 'TBSHT01' substring made the segmenter decide that there was also a unit there to be classified, and so the unit classifier also ran. However, it would not have seen data like this before so it returned the list of results it regularly returns when it receives an input it's never seen.

Wind Speed m/s
- "wind speed" is one of the few class labels that the classifier has more than two samples for, and it shows. Wind speed is correctly identified as the propery class, with m/s being correctly identified as the "meter per second" unit class in 5th place. 

AvgWindDirection
- While the program doesn't have a way of distinguishing qualifier words such as "average", "maximum", etc, it is not an issue in this case. "Wind direction" is correctly identified as the property class in first place.

MinWindSpeed
- While "wind speed" is correctly identified as the property class, "min" is incorrectly identified as unit and classified as such, due the the above mentioned issue of not treating qualifier words seperately.

### 1.2 Simple data

The program performs better when input is more in line with what it has ingested through training. The classifier training sets are my_property.csv and my_unit.csv, view them for examples of possible inputs and labels. 

The simple data we will try to match now:
- wind speed km per s
- AUD
- dollar
- temp degC
- carbon concentration

In [11]:
user_input_loop(notebook=True)

Enter a string to predict:wind speed km per s


,processed_name,proper_name,suggested_unit,suggested_unit_url,url
0,wind speed,wind speed,Kilometer per Hour,http://registry.it.csiro.au/def/qudt/1.1/qudt-...,http://registry.it.csiro.au/def/environment/pr...
1,wind direction,wind direction,Degree Angle,http://registry.it.csiro.au/def/qudt/1.1/qudt-...,http://registry.it.csiro.au/def/environment/pr...
2,maximum wind gust,maximum wind gust,NaN,NaN,NaN
3,wind velocity,wind velocity,NaN,NaN,NaN
4,carbon concentration,carbon concentration,NaN,NaN,http://registry.it.csiro.au/def/environment/pr...
5,oxygen concentration,oxygen concentration,NaN,NaN,http://registry.it.csiro.au/def/environment/pr...
6,water temperature,water temperature,Degree Celsius,http://qudt.org/vocab/unit#DegreeCelsius,http://registry.it.csiro.au/def/environment/pr...
7,nitrogen concentration,nitrogen concentration,Milligrams Per Litre | Milligrams Per Cubic Me...,http://registry.it.csiro.au/def/environment/un...,http://registry.it.csiro.au/def/environment/pr...
8,hydrocarbon concentration,hydrocarbon concentration,NaN,NaN,http://registry.it.csiro.au/def/environment/pr...
9,phosphorus concentration,phosphorus concentration,Milligrams Per Litre,http://registry.it.csiro.au/def/environment/un...,http://registry.it.csiro.au/def/environment/pr...


,abbreviation,processed_name,proper_name,url
0,km,kilometer,Kilometer,http://registry.it.csiro.au/def/qudt/1.1/qudt-...
1,km/s,kilometer per second,Kilometer per Second,http://registry.it.csiro.au/def/qudt/1.1/qudt-...
2,km/hr,kilometer per hour,Kilometer per Hour,http://registry.it.csiro.au/def/qudt/1.1/qudt-...
3,km^3/s^2,cubic kilometer per second squared,Cubic Kilometer per Second Squared,http://registry.it.csiro.au/def/qudt/1.1/qudt-...
4,J/(km-K-Pa),joule per kilogram kelvin per pascal,Joule per Kilogram Kelvin per Pascal,http://registry.it.csiro.au/def/qudt/1.1/qudt-...
5,s,second,Second,http://registry.it.csiro.au/def/qudt/1.1/qudt-...
6,s^-1,inverse second,Inverse Second,http://registry.it.csiro.au/def/qudt/1.1/qudt-...
7,S,siemens,Siemens,http://registry.it.csiro.au/def/qudt/1.1/qudt-...
8,m s^-1,meter per second,Meter per Second,http://registry.it.csiro.au/def/qudt/1.1/qudt-...
9,ft/s,foot per second,Foot per Second,http://registry.it.csiro.au/def/qudt/1.1/qudt-...


-----------------------------------------------------------
Enter a string to predict:AUD


,abbreviation,processed_name,proper_name,url
0,AUD,australian dollar,Australian Dollar,http://registry.it.csiro.au/def/qudt/1.1/qudt-...
1,m,meter,Meter,http://registry.it.csiro.au/def/qudt/1.1/qudt-...
2,s,second,Second,http://registry.it.csiro.au/def/qudt/1.1/qudt-...
3,m^-3,per cubic meter,Per Cubic Meter,http://registry.it.csiro.au/def/qudt/1.1/qudt-...
4,ft,foot,Foot,http://registry.it.csiro.au/def/qudt/1.1/qudt-...
5,m s^-1,meter per second,Meter per Second,http://registry.it.csiro.au/def/qudt/1.1/qudt-...
6,m^2,square meter,Square Meter,http://registry.it.csiro.au/def/qudt/1.1/qudt-...
7,NaN,inch per second,Inch per Second,http://registry.it.csiro.au/def/qudt/1.1/qudt-...
8,NaN,degree per minute,Degree per Minute,http://registry.it.csiro.au/def/qudt/1.1/qudt-...
9,ft/s,foot per second,Foot per Second,http://registry.it.csiro.au/def/qudt/1.1/qudt-...


...

,abbreviation,processed_name,proper_name,url
0,degC,degree celsius,Degree Celsius,http://registry.it.csiro.au/def/qudt/1.1/qudt-...
1,degC / m,degree celsius per minute,Degree Celsius per Minute,http://registry.it.csiro.au/def/qudt/1.1/qudt-...
2,degC / s,degree celsius per second,Degree Celsius per Second,http://registry.it.csiro.au/def/qudt/1.1/qudt-...
3,cm-degC,degree celsius centimeter,Degree Celsius Centimeter,http://registry.it.csiro.au/def/qudt/1.1/qudt-...
4,degC / hr,degree celsius per hour,Degree Celsius per Hour,http://registry.it.csiro.au/def/qudt/1.1/qudt-...
5,mol-degC,mole degree celsius,Mole Degree Celsius,http://registry.it.csiro.au/def/qudt/1.1/qudt-...
6,cm-s-degC,centimeter second degree celsius,Centimeter Second Degree Celsius,http://registry.it.csiro.au/def/qudt/1.1/qudt-...
7,g-degC,gram degree celsius,Gram Degree Celsius,http://registry.it.csiro.au/def/qudt/1.1/qudt-...
8,kcal/(mol-degC),kilocalorie per mole degree celsius,Kilocalorie per Mole Degree Celsius,http://registry.it.csiro.au/def/qudt/1.1/qudt-...
9,kcal/(gm-degC),calorie per gram degree celsius,Calorie per Gram Degree Celsius,http://registry.it.csiro.au/def/qudt/1.1/qudt-...


-----------------------------------------------------------
Enter a string to predict:carbn concentration
no abbreviation for carbn


,processed_name,proper_name,suggested_unit,suggested_unit_url,url
0,carbon concentration,carbon concentration,NaN,NaN,http://registry.it.csiro.au/def/environment/pr...
1,carbon organic concentration,carbon - organic concentration,NaN,NaN,http://registry.it.csiro.au/def/environment/pr...
2,carbon inorganic concentration,carbon - inorganic concentration,NaN,NaN,http://registry.it.csiro.au/def/environment/pr...
3,carbon dissolved organic concentration,carbon - dissolved organic concentration,NaN,NaN,http://registry.it.csiro.au/def/environment/pr...
4,carbon 14 concentration,carbon 14 concentration,Percent,http://qudt.org/vocab/unit#Percent,http://registry.it.csiro.au/def/environment/pr...
5,carbon monoxide concentration,carbon monoxide concentration,NaN,NaN,http://registry.it.csiro.au/def/environment/pr...
6,carbon disulfide concentration,carbon disulfide concentration,Milligrams Per Litre,http://registry.it.csiro.au/def/environment/un...,http://registry.it.csiro.au/def/environment/pr...
7,carbon 13 concentration,carbon 13 concentration,Percent,http://qudt.org/vocab/unit#Percent,http://registry.it.csiro.au/def/environment/pr...
8,carbon suspended total,carbon - suspended total,NaN,NaN,http://registry.it.csiro.au/def/environment/pr...
9,carbon dissolved inorganic concentration,carbon - dissolved inorganic concentration,NaN,NaN,http://registry.it.csiro.au/def/environment/pr...


""


-----------------------------------------------------------
Enter a string to predict:xxx


#### Summary of simple data results

wind speed km per s
- The property "wind speed" is correctly found in 1st place, and the unit "kilometer per second" is found in 2nd place. Labels which contain some of the same words are rated highly.

AUD
- Currencies are included, but other dollar-based currencies are not included in the results as the classifier is not aware that AUD is actually equal to the words "Australian", "dollar". There is no transformation from the abbreviation - it's treated as just another word. Therefore, the rest of the results are not useful and reflect only probabilities when there is no understandable input.

dollar
- A list of units which are dollars is returned. Unfortunately, not all terms are included in this, as there are more than 10.

temp degC
- "Air temperature" is guessed as the most likely candidate for the term "temperature". Degree celsius is also correctly identified.
- "temp" is a common shortening of "temperature", so any instances of it are altererd in the solve_abbreviations method of preprocessing.py. It contains a dictionary of abbreviations and replacements, which can be extended. 

carbn concentration
- As there are many terms which "carbon concentration" can refer to, the most likely are returned for you to choose.
- The program can deal with terms which are slightly misspelled or use a regional variant such as meter=metre. This is governed by the solve_similar_spelling method in preprocessing.py - useful to take a look at if you wonder why your misspelling isn't being correctly corrected.

## 2. Comparison with rule-based project

The previous conducted with the same premise but a rule-based approach is located at https://github.com/cherrycheng1202/String-Matching-Scientific-Variables. Its results will be compared against the current, machine learning approach. 

As I had some issues running it from this notebook, input and results will be pasted in.

Note that this approach did not perform automatic segmentation, and requested the user to input Name ("property") and unit seperately.

Name: temp  
value:  
unit: degC  
1 soil temperature in Degree Celsius [ soil temperature (http://registry.it.csiro.au/sandbox/environment/property/soil_temperature) Degree Celsius (http://registry.it.csiro.au/def/qudt/1.1/qudt-unit/DegreeCelsius) ]  
2 air temperature in Degree Celsius [ air temperature (http://registry.it.csiro.au/def/environment/property/air_temperature) Degree Celsius (http://registry.it.csiro.au/def/qudt/1.1/qudt-unit/DegreeCelsius) ]  
3 dew point temperature in Degree Celsius [ dew point temperature (http://registry.it.csiro.au/def/environment/property/dew_point_temperature) Degree Celsius (http://registry.it.csiro.au/def/qudt/1.1/qudt-unit/DegreeCelsius) ]  
4 sea water temperature in Degree Celsius [ sea water temperature (http://registry.it.csiro.au/def/environment/property/sea_water_temperature) Degree Celsius (http://registry.it.csiro.au/def/qudt/1.1/qudt-unit/DegreeCelsius) ]  
5 water temperature in Degree Celsius [ water temperature (http://registry.it.csiro.au/def/environment/property/water_temperature) Degree Celsius (http://registry.it.csiro.au/def/qudt/1.1/qudt-unit/DegreeCelsius) ]

Name: wind speed  
value:   
unit: m/s  
No result

Name: carbn concentration  
value:  
unit:  
No result

Name: AvgWindDirection  
value:   
unit:   
1 Average wind direction in Degree Angle [ wind direction (http://registry.it.csiro.au/def/environment/property/_wind_direction) Degree Angle (http://registry.it.csiro.au/def/qudt/1.1/qudt-unit/DegreeAngle) ]

Name: carbon concentration  
value:   
unit:   
1 calcium carbonate concentration [ calcium carbonate concentration   (http://registry.it.csiro.au/def/environment/property/calcium_carbonate_concentration) none () ]  
2 carbon 13 concentration in Percent [ carbon 13 concentration   (http://registry.it.csiro.au/def/environment/property/carbon_13_concentration) Percent   (http://registry.it.csiro.au/def/qudt/1.1/qudt-unit/Percent) ]  
3 carbon 14 concentration in Percent [ carbon 14 concentration   (http://registry.it.csiro.au/def/environment/property/carbon_14_concentration) Percent   (http://registry.it.csiro.au/def/qudt/1.1/qudt-unit/Percent) ]   
4 carbon concentration [ carbon concentration (http://registry.it.csiro.au/sandbox/soil/soil-property/carbon_concentration) none () ]  
5 total dissolved carbon dioxide concentration in Mole Percent [ total dissolved carbon dioxide concentration   (http://registry.it.csiro.au/def/environment/property/carbon_dioxide_concentration) Mole Percent   (http://registry.it.csiro.au/def/environment/unit/_MolPercent) ]  
6 total dissolved carbon dioxide concentration in Percent [ total dissolved carbon dioxide concentration   (http://registry.it.csiro.au/def/environment/property/carbon_dioxide_concentration) Percent   (http://registry.it.csiro.au/def/qudt/1.1/qudt-unit/Percent) ]  
7 total dissolved carbon dioxide concentration in MilliGrams per Litre [ total dissolved carbon dioxide concentration   (http://registry.it.csiro.au/def/environment/property/carbon_dioxide_concentration) MilliGrams per Litre   (http://registry.it.csiro.au/def/environment/unit/_MilligramsPerLitre) ]  
8 carbon - dissolved inorganic concentration [ carbon - dissolved inorganic concentration   (http://registry.it.csiro.au/def/environment/property/carbon_dissolved_inorganic_concentration) none () ]  
9 carbon - dissolved organic concentration [ carbon - dissolved organic concentration   (http://registry.it.csiro.au/def/environment/property/carbon_dissolved_organic_concentration) none () ]  
10 carbon disulfide concentration in MilliGrams per Litre [ carbon disulfide concentration   (http://registry.it.csiro.au/def/environment/property/carbon_disulfide_concentration) MilliGrams per Litre   (http://registry.it.csiro.au/def/environment/unit/_MilligramsPerLitre) ]  
11 carbon - inorganic concentration [ carbon - inorganic concentration   (http://registry.it.csiro.au/def/environment/property/carbon_inorganic_concentration) none () ]  
12 carbon monoxide concentration [ carbon monoxide concentration   (http://registry.it.csiro.au/def/environment/property/carbon_monoxide_concentration) none () ]  
13 carbon - organic concentration [ carbon - organic concentration   (http://registry.it.csiro.au/def/environment/property/carbon_organic_concentration) none () ]  
14 carbon - suspended inorganic concentration [ carbon - suspended inorganic concentration   (http://registry.it.csiro.au/def/environment/property/carbon_suspended_inorganic_concentration) none () ]   
15 carbon - suspended organic concentration [ carbon - suspended organic concentration   (http://registry.it.csiro.au/def/environment/property/carbon_suspended_organic_concentration) none () ]  
16 carbonate concentration [ carbonate concentration   (http://registry.it.csiro.au/def/environment/property/carbonate_concentration) none () ]  
17 calcium carbonate concentration in Percent [ calcium carbonate concentration (http://registry.it.csiro.au/sandbox/soil/soil-property/calcium_carbonate_concentration) Percent (http://registry.it.csiro.au/def/qudt/1.1/qudt-unit/Percent) ]  
18 carbon - organic concentration in Percent [ carbon - organic concentration (http://registry.it.csiro.au/sandbox/soil/soil-property/carbon_organic_concentration) Percent (http://registry.it.csiro.au/def/qudt/1.1/qudt-unit/Percent) ]  
19 carbonate concentration in Percent [ carbonate concentration (http://registry.it.csiro.au/sandbox/soil/soil-property/carbonate_concentration) Percent (http://registry.it.csiro.au/def/qudt/1.1/qudt-unit/Percent) ]  

#### Summary of rule-based results

The rule-based project has the benefit of being able to display a list of results as long as it likes, as it limits those results strictly. All results that are returned are relevant. 

However, it does not deal with misspellings. Its methods of assessing input is also stricter, and it has to be in a format that it is familiar with to return results. This is interesting, as my program did not transform input in any machine learning way - but it did, so that it could be more flexible.

I think that something valuable could be made of integrating the best parts of the two projects. Input processing, input classification/segmentation from the ML project, and results selection and filtering from the RB project. This mixed approach would maximise the strong points of the seperate approaches while minimising their weaknesses.

## 3. Suggestions for future improvement

Treating qualifier words
- By adding a method that identifies 'qualifier' words (eg. maximum, minimum, total, average, mean) which may appear in any property term but only affect its limit and not its intrinsic meaning, and deals with them seperately. 

- Since these words are limited, they could be easily be all stored in a dictionary. 

- I don't think it would be a good idea to create seperate versions of property classes which only differed in containing a qualifier word (eg. "average wind speed" vs the current "wind speed"), as even though that could be more informative, it would be difficult to implement. Classes which could exist with these qualifier words would have to be identified, the number of permutations and therefore new classes would be high, and at least one sample of training data would have to be created for each new class. However, if it was genuinely more informative to do this, and only for a number of classes that would benefit from it, it absolutely could.

- Problem: qualifier words are sometimes already part of a property name. In these cases, they do not just limit the property, but are an intrinsic part of it. For example, "MSLP", or, "mean sea level pressure"; "total solids"

- Suggestion: remove qualifier words from the input string once identified, and append onto the output string when returning results.

Extension of training data
- The classifier probabilities are currently very sensitive as each class has a very small number of samples. More sample data for each class would increase confidence and accuracy.
- If more class labels and data were found, they could be added to the training sets to give the classifier a broader scope in what labels it could assign.
- Suggestion: to increase accuracy, add more data samples. To extend scope, add more classes.

Improve input segmentation
- Currently, input words are classified as "property" or "unit", then sent to those respective classifiers to be classified more specifically. The machine learning model which performs the binary classification is a recurrent neural network with an LSTM (long short term memory) layer, which allows it to retain the knowledge of the order in which it receives words. However, this is currently not relevant as it only receives one word at a time (windowsize = 1). If the windowsize were changed to 2, it could more correctly classify words which are usually "unit" (or "property"), but in that context belong to "property". For example, "chrorophyll a concentration" is all one property, but when segmented one-by-one, "a" would be categorised as a unit.
- This has not been implemented as the current method for segmentation is fairly simple, and sends the words in the property and unit sets to their respective classifiers. However, with two words receiving the same class at a time, words which were at a boundary of property and unit phrases could be sent to the wrong classifier. For example, given, "water temp degree celsius", with a windowsize=2, the inputs to the segmenter would be: ["water temp", "temp degree", "degree celcius"]. And the inputs to the classifiers would be something like: property['water', 'temp', 'degree']; unit['temp', 'degree', 'celsius'] - overlapping. This could just be noisy, or could contribute to misclassification.
- Suggestion: modify the segment_user_string method in main.py to use a more complex algorithm and a window_size higher than 1. 

Integrate user-created data into models
- On the web application, there is currently a form into which a user can enter input strings and select their matching classes, to have that data added to a file which can be included in the training set. However, integration for this is not yet supported as it is very easy to create bad data, and it would all need to be validated first.
- Suggestion: if the web application is hosted and becomes used, the user_trainingset.csv file can be added to the files which go through the raw_to_clean function in preprocessing.py to become part of the training set.

## 4 Conclusions

The most hindering aspect of the project is the data that is available to it. As it uses supervised machine learning methods, it is utterly reliant on labelled data and would prefer to have a high volume of it. Unfortunately, the data I was able to find to support this was not as comprehensive as I would have hoped. 